In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
case_oppertunity = pd.read_csv('/Users/michaelcolellajensen/Desktop/Tableau/case_oppertunities.csv')
finance_grouped = pd.read_csv('/Users/michaelcolellajensen/Downloads/finance_fact.csv')
finance_grouped['MONTH_END_DATE'] = pd.to_datetime(finance_grouped['MONTH_END_DATE'])
finance_recent = finance_grouped.sort_values('MONTH_END_DATE', ascending=False).drop_duplicates('ACCOUNTKEY')

In [4]:
recent_opp_act = case_oppertunity.sort_values('CLOSEDATE', ascending=False).drop_duplicates('ACCOUNTKEY')
recent_opp_act['total_acv'] = recent_opp_act[['Einstein Analytics', 'Role Based - Online',
                                              'Server', 'Saas', 'Role Based - On Prem',
                                              'Services', 'Desktop', 'Role Based - NonProd']].sum()
print(recent_opp_act.shape)
recent_opp_act = recent_opp_act.merge(finance_recent,
                                      how='left',
                                      on='ACCOUNTKEY')
print(recent_opp_act.shape)

(28348, 36)


ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [ ]:
recent_opp_binary = recent_opp_act
recent_opp_act['Einstein Analytics'] = np.where(recent_opp_act['Einstein Analytics'] > 0, 1, 0)
recent_opp_act['Server'] = np.where(recent_opp_act['Server'] > 0, 1, 0)
recent_opp_act['Saas'] = np.where(recent_opp_act['Saas'] > 0, 1, 0)
recent_opp_act['Role Based - On Prem'] = np.where(recent_opp_act['Role Based - On Prem'] > 0, 1, 0)
recent_opp_act['Role Based - Online'] = np.where(recent_opp_act['Role Based - Online'] > 0, 1, 0)
recent_opp_act['Services'] = np.where(recent_opp_act['Services'] > 0, 1, 0)
recent_opp_act['Desktop'] = np.where(recent_opp_act['Desktop'] > 0, 1, 0)
recent_opp_act['Role Based - NonProd'] = np.where(recent_opp_act['Role Based - NonProd'] > 0, 1, 0)
recent_opp_act['Other'] = np.where(recent_opp_act['Other'] > 0, 1, 0)
print(recent_opp_act.head())

In [ ]:
def assign_cohort(x):
    if x['Tenure'] <= 1:
        return 4
    elif x['Tenure'] <= 5:
        return 3
    elif x['Tenure'] <= 9:
        return 3
    elif x['Tenure'] <= 15:
        return 2
    elif x['Tenure'] <= 25:
        return 1
    else:
        return 0
    
recent_opp_binary['cohort'] = recent_opp_binary.apply(assign_cohort,
                                                axis=1)
print(recent_opp_binary.shape)
cohort_size = [len(recent_opp_act[recent_opp_act['cohort'] == 0]),
               len(recent_opp_act[recent_opp_act['cohort'] == 1]),
               len(recent_opp_act[recent_opp_act['cohort'] == 2]),
               len(recent_opp_act[recent_opp_act['cohort'] == 3]),
               len(recent_opp_act[recent_opp_act['cohort'] == 4])]

In [ ]:
summary = recent_opp_act.groupby('cohort').sum(['Einstein Analytics', 'Role Based - Online',
                                              'Server', 'Saas', 'Role Based - On Prem',
                                              'Services', 'Desktop', 'Role Based - NonProd'])
#summary.drop(['Unnamed: 0', 'STATUS', 'PRIORITY', 'IMPACT', 'ORIGIN', 'PRODUCT', 'CATEGORY',
              #'SUBCATEGORY', 'CREATED_DATE', 'CLOSED_DATE', 'CLOSEDATE', 'ISWON', 'CASE_NUMBER',
              #'ISPARTNERRELATED', 'total_acv', 'CASE_CLOSED_REASON', 'RECORDTYPE',
              #'ACCOUNTTYPE', 'ACCOUNTTYPEDETAIL', 'EMPLOYEECOUNT', 'THEATER', 'SEGMENT',
              #'SUBREGION', 'SUBSEGMENT', 'Tenure', 'Zero.Revenue.Flag'], inplace = True, axis = 1)
summary.drop(['Unnamed: 0', 'EMPLOYEECOUNT', 'Tenure', 'Zero.Revenue.Flag',
              'ISWON', 'ISPARTNERRELATED', 'total_acv'], inplace = True, axis = 1)
print(summary)

In [ ]:
product_use = summary.divide(cohort_size, axis=0)
#product_use.drop('id', axis=1, inplace=True)
product_use.round(decimals=3)
plt.figure(figsize=(10,8))
plt.title('Product Useage')
sns.heatmap(data=product_use, 
            annot= True,
            fmt='.0%',
            vmin=0.0,
            vmax=0.18,
            cmap='PuBu')
plt.show()

In [ ]:
cases_grouped = case_oppertunity.groupby('ACCOUNTKEY')['CASE_NUMBER'].agg(['count'])
print(cases_grouped)
print(cases_grouped['count'].mean())

In [ ]:
account_case_count = recent_opp_binary.merge(cases_grouped,
                                             on='ACCOUNTKEY',
                                             how='inner')
print(account_case_count.shape)

In [ ]:
def assign_case_cohort(x):
    if x['count'] <= 1:
        return 5
    elif x['count'] <= 4:
        return 4
    elif x['count'] <= 9:
        return 3
    elif x['count'] <= 30:
        return 2
    elif x['count'] <= 100:
        return 1
    else:
        return 0
    
account_case_count['case_cohort'] = account_case_count.apply(assign_case_cohort,
                                                             axis=1)
account_case_count = account_case_count[['case_cohort', 'Einstein Analytics', 
                                         'Role Based - Online','Server', 'Saas', 
                                         'Role Based - On Prem','Services', 'Desktop', 
                                         'Role Based - NonProd']]
cohort_case_size = [len(account_case_count[account_case_count['case_cohort'] == 0]),
               len(account_case_count[account_case_count['case_cohort'] == 1]),
               len(account_case_count[account_case_count['case_cohort'] == 2]),
               len(account_case_count[account_case_count['case_cohort'] == 3]),
               len(account_case_count[account_case_count['case_cohort'] == 4]),
               len(account_case_count[account_case_count['case_cohort'] == 5])]
print(cohort_size)

In [ ]:
case_summary = account_case_count.groupby('case_cohort').sum(['Einstein Analytics', 'Role Based - Online',
                                                          'Server', 'Saas', 'Role Based - On Prem',
                                                          'Services', 'Desktop', 'Other'])
case_summary.drop('count', inplace=True, axis=1)
product_use_by_case = case_summary.divide(cohort_case_size, axis=0)
#product_use.drop('id', axis=1, inplace=True)
product_use_by_case.round(decimals=3)
plt.figure(figsize=(10,8))
plt.title('Product Useage by Case Cohort')
sns.heatmap(data=product_use_by_case, 
            annot= True,
            fmt='.0%',
            vmin=0.0,
            vmax=0.50,
            cmap='PuBu')
plt.show()
